In [1]:
import pandas as pd

# Show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# If needed, also expand the width
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)  # or a large int

from copy import deepcopy
from random import randint, choice
import random
import numpy as np

# from library.solution import Solution
# from library.algorithms.hill_climbing import hill_climbing
# from library.algorithms.simulated_annealing import simulated_annealing
from library.wedding_solution_list import Wedding_Solution

In [ ]:
# from library.solution import Solution
# import pandas as pd
# # from random import randint
# # import random
# fitness_grid = pd.read_csv("library/wedding_seat_data.csv")

# class Wedding_Solution(Solution):
#     def __init__(self, repr=None, tables=8, attendees=64, values_grid=fitness_grid):
#         self.tables = tables
#         self.attendees = attendees
#         self.values_grid = values_grid
#         self.seats_per_table = int(attendees / tables)

#         if repr:
#             repr = self._validate_repr(repr)

#         super().__init__(repr=repr)

#     # only used to validate the given representation (random initialization already follows these rules)
#     def _validate_repr(self, repr):
#         # repr needs to be a dictionary
#         if not isinstance(repr, list):
#             raise TypeError("Representation must be a list.")
        
#         # this ensures that the partitions are always even and the same amounts
#         if not (self.attendees / self.tables).is_integer():
#             raise ValueError("The number of attendees and tables for this solution must divide evenly into each other.")
        
#         # all the values in the repr need to be an int, if not - change to an int
#         if not all([isinstance(table_num, int) for table_num in repr]):
#             repr = [int(table) for table in repr]

#         # the number of unique numbers in repr need to be the same as the number of tables defined
#         if len(set(repr)) != self.tables:
#             raise ValueError("Missing a table number from representation assignment: Table(s)", set(range(1,self.tables+1)) - set(repr))
        
#         # there needs to be the same number of ppl at each table
#         if not all(repr.count(x) == self.seats_per_table for x in range(1,self.tables)):
#             raise ValueError("The number of people assigned to each table need to be the same.")
        
#         # the length of all the attendees need to match the defined variable for attendees
#         if len(repr) != self.attendees:
#             raise ValueError("The total number of attendees in the representation need to match the defined number of attendees")
        
#         return repr

#     # Override the superclass's methods
#     def random_initial_representation(self):
#         seats = []
#         for i in range(1,self.tables+1): # generates the same number of numbers in a row ([1] * 5 = [1,1,1,1,1])
#             seats = seats + [i] * self.seats_per_table
#         random.shuffle(seats) # randomly assign tables to participants
#         return seats

#     # returns the fitness of the representation / solution
#     def fitness(self):
#         fitness = 0
#         for table_num in range(1, self.tables+1): # loop through the tables
#             fitness += self.table_fitness(table_num)

#         return fitness
    
#     # checks the fitness of a specific table instead of all the tables
#     def table_fitness(self, table):
#         fitness = 0
#         table_seats = [i for i, x in enumerate(self.repr) if x == table]
#         table_seats = [x + 1 for x in table_seats] # add 1 to everything to match the fitness grid
#         people_seen = []
#         for personA in table_seats: # loop through the people at the table 
#             people_seen.append(personA) # make sure to skip personA's score with themself
#             for personB in table_seats: # compare person A with everyone else (personB) at the table
#                 if personB not in people_seen: # skip the people that were already counted to not count them twice
#                     fitness += self.values_grid[self.values_grid['idx'] == personA][str(personB)].values[0]
#         return fitness
    
#     # clearly shows which person sits at which table for debugging purposes
#     def pretty_print(self):
#         tables = {i: [] for i in range(1, self.tables+1)}  # Tables 1 through 8

#         for idx, table in enumerate(self.repr):
#             tables[table].append(idx)

#         print("----------SEATING ARRANGEMENTS----------")
#         for table_num in range(1, self.tables+1): 
#             # checks to see if its single digit - if so, add a space for cleaner reading
#             # also adds 1 so that it is clear which person is being referred to
#             indices = [f"{i+1:2}" for i in tables[table_num]]
#             print(f"Table {table_num}: [{', '.join(indices)}]")
#         print("----------------------------------------")

In [2]:
tables = 4
attendees = 12
initial_solution = [1, 3, 1, 1, 4, 2, 4, 4, 3, 3, 2, 2]
repr_initial_solution = Wedding_Solution(repr=initial_solution, tables=tables, attendees=attendees)
print("ASSIGNED INITIALIZATION: ")
print("Repr: ", repr_initial_solution)
print("Seats per table: ", repr_initial_solution.seats_per_table)
print("Fitness: ", repr_initial_solution.fitness())

print("\nRANDOM INITIALIZATION: ")
repr = Wedding_Solution(tables=tables, attendees=attendees)
print("Repr: ", repr)
print("Seats per table: ", repr.seats_per_table)
print("Fitness: ", repr.fitness())

ASSIGNED INITIALIZATION: 
Repr:  [1, 3, 1, 1, 4, 2, 4, 4, 3, 3, 2, 2]
Seats per table:  3
Fitness:  7000

RANDOM INITIALIZATION: 
Repr:  [2, 1, 3, 3, 4, 1, 3, 2, 1, 4, 2, 4]
Seats per table:  3
Fitness:  3400


In [3]:
class WeddingGASolution(Wedding_Solution):
    def __init__(
        self,
        mutation_function,
        crossover_function,
        tables,
        attendees,
        values_grid,
        repr = None
    ):
        # Save as attributes for access in methods
        self.mutation_function = mutation_function
        self.crossover_function = crossover_function

        super().__init__(
            repr=repr,
            tables=tables,
            attendees=attendees,
            values_grid=values_grid,
        )

    def mutation(self, mut_prob):
        # Apply mutation function to representation
        new_repr = self.mutation_function(self.repr, mut_prob)
        # Create and return individual with mutated representation
        return WeddingGASolution(
            selection_function = self.selection_function,
            mutation_function=self.mutation_function,
            crossover_function=self.crossover_function,
            repr=new_repr
        )

    def crossover(self, other_solution):
        # Apply crossover function to self representation and other solution representation
        offspring1_repr, offspring2_repr = self.crossover_function(self.repr, other_solution.repr)

        # Create and return offspring with new representations
        return (
            WeddingGASolution(
                selection_function = self.selection_function,
                mutation_function=self.mutation_function,
                crossover_function=self.crossover_function,
                repr=offspring1_repr
            ),
            WeddingGASolution(
                selection_function = self.selection_function,
                mutation_function=self.mutation_function,
                crossover_function=self.crossover_function,
                repr=offspring2_repr
            )
        )

In [3]:
repr = Wedding_Solution()
print(repr)
repr.pretty_print()

[6, 1, 2, 8, 5, 8, 6, 7, 3, 7, 4, 5, 1, 8, 2, 5, 4, 5, 6, 5, 6, 3, 2, 8, 8, 4, 3, 3, 6, 8, 2, 3, 4, 7, 7, 8, 1, 7, 6, 7, 2, 8, 6, 4, 1, 4, 2, 1, 1, 6, 1, 2, 3, 3, 4, 2, 7, 7, 5, 5, 1, 5, 4, 3]
----------SEATING ARRANGEMENTS----------
Table 1: [ 2, 13, 37, 45, 48, 49, 51, 61]
Table 2: [ 3, 15, 23, 31, 41, 47, 52, 56]
Table 3: [ 9, 22, 27, 28, 32, 53, 54, 64]
Table 4: [11, 17, 26, 33, 44, 46, 55, 63]
Table 5: [ 5, 12, 16, 18, 20, 59, 60, 62]
Table 6: [ 1,  7, 19, 21, 29, 39, 43, 50]
Table 7: [ 8, 10, 34, 35, 38, 40, 57, 58]
Table 8: [ 4,  6, 14, 24, 25, 30, 36, 42]
----------------------------------------


In [4]:
mut_prob = 0.5
x = 0.62#random.random()

# if x <= mut_prob:
#     print("Entered.")
# else:
#     print("Didn't Enter.")

if x > mut_prob:
    print("Didn't Enter.")
else:
    print("Entered.")

print("Chosen x: ", round(x,2))

Didn't Enter.
Chosen x:  0.62


### Selection Algorithms

- Fitness Proportionate Selection <span style="color:red">(from class)</span>
- Ranking
- Tournament

### Ranking Selection

<span style="color:red">[CHANGE] Get from textbook - currently chatgpt</span>
- Evaluate fitness of all individuals in the population.

- Sort individuals from best to worst based on fitness.

- Assign ranks (e.g., rank 1 for worst, rank N for best).

- Convert ranks into selection probabilities

 ![Ranking](ranks.png)

Or use linear ranking or exponential ranking schemes to control selection pressure.

- Select individuals based on those probabilities

In [5]:
def ranking_selection(population, maximization):
    # evalutate the fitness and get the total fitness
    total_fitness = 0
    fitness_list = [] # list of tuples (ind, fitness)
    for indiv in population:
        fitness = indiv.fitness()
        fitness_list.append((indiv, fitness))
        total_fitness += fitness

    # sort by fitness (if max, sort in desc, otherwise, asc)
    if maximization:
        sorted(fitness_list, key=lambda x: x[1], reverse=True)
    else:   
        sorted(fitness_list, key=lambda x: x[1])
            
    # ranks will the the index in the list so index 0 will have rank 1
    # converting ranks to probabilities - the index of the rank list will match the index of the fitness_list
    rank_prob_list = []
    rank_summation = sum(list(range(1,65)))
    for index, indiv in enumerate(fitness_list):
        rank = index+1
        rank_prob = rank / rank_summation
        rank_prob_list.append(rank_prob)

    # select indiv
    selection = random.choices(fitness_list, weights=rank_prob_list)
    
    print("Selected individual " + str(selection))
    selection[0][0].pretty_print()

    return selection


In [6]:
population = []
fitness_list = []
for _ in range(1, 10):
    x = Wedding_Solution()
    population.append(x)
    fitness_list.append(x.fitness())

print(fitness_list)
print("--------")
selected = ranking_selection(population=population, maximization=True)

[6300, 12300, 11200, 17700, 12300, 10700, 9600, 14500, 14000]
--------
Selected individual [([4, 5, 6, 2, 7, 8, 6, 4, 1, 6, 3, 4, 5, 7, 2, 2, 5, 1, 7, 8, 5, 6, 7, 8, 8, 1, 7, 5, 6, 6, 2, 2, 5, 3, 3, 1, 1, 7, 2, 5, 4, 2, 3, 4, 5, 3, 4, 6, 4, 7, 8, 3, 7, 3, 8, 8, 6, 2, 1, 4, 8, 1, 3, 1], 14000)]
----------SEATING ARRANGEMENTS----------
Table 1: [ 9, 18, 26, 36, 37, 59, 62, 64]
Table 2: [ 4, 15, 16, 31, 32, 39, 42, 58]
Table 3: [11, 34, 35, 43, 46, 52, 54, 63]
Table 4: [ 1,  8, 12, 41, 44, 47, 49, 60]
Table 5: [ 2, 13, 17, 21, 28, 33, 40, 45]
Table 6: [ 3,  7, 10, 22, 29, 30, 48, 57]
Table 7: [ 5, 14, 19, 23, 27, 38, 50, 53]
Table 8: [ 6, 20, 24, 25, 51, 55, 56, 61]
----------------------------------------


### Tournament Selection

<span style="color:red">[CHANGE] Get from textbook - currently chatgpt</span>

- Randomly select k individuals from the population (the "tournament").

- Evaluate their fitness.

- Choose the best one from that group to be a parent.

- Repeat until you've selected as many parents as needed.

In [7]:
def tournament_selection(population, maximization):
    # randomly select the number of indivs to compare
    tourney_indivs_num = randint(2, int(len(population)/2)) # limiting range to half the population size
    tourney_indivs_start_idx = randint(0, len(population)-tourney_indivs_num)
    tourney_indivs_end_idx = tourney_indivs_start_idx + tourney_indivs_num
    tourney_indivs = population[tourney_indivs_start_idx:tourney_indivs_end_idx]

    print("Range: " + str(tourney_indivs_start_idx) + " - " + str(tourney_indivs_end_idx))

    # get the subset's fitnesses
    fitness_list = [(ind, ind.fitness()) for ind in tourney_indivs]

    # select the best individual (if max, get the max(), otherwise, get the min())
    if maximization:
        selection = max(fitness_list, key=lambda x: x[1])
    else:
        selection = min(fitness_list, key=lambda x: x[1])

    print("Selected individual " + str(selection))
    selection[0].pretty_print()
    
    return selection

In [8]:
population = []
fitness_list = []
for _ in range(1, 10):
    x = Wedding_Solution()
    population.append(x)
    fitness_list.append(x.fitness())

print(fitness_list)
print("--------")
selected = tournament_selection(population=population, maximization=True)

[12800, 8400, 13500, 14400, 9700, 13600, 17900, 11700, 8800]
--------
Range: 1 - 3
Selected individual ([4, 8, 4, 1, 3, 5, 1, 2, 1, 5, 7, 7, 4, 2, 1, 4, 6, 8, 2, 3, 5, 6, 4, 4, 3, 7, 5, 8, 2, 5, 8, 1, 7, 6, 4, 1, 1, 8, 5, 5, 2, 6, 8, 2, 3, 8, 7, 3, 5, 6, 7, 4, 2, 6, 3, 3, 3, 1, 6, 7, 2, 6, 8, 7], 13500)
----------SEATING ARRANGEMENTS----------
Table 1: [ 4,  7,  9, 15, 32, 36, 37, 58]
Table 2: [ 8, 14, 19, 29, 41, 44, 53, 61]
Table 3: [ 5, 20, 25, 45, 48, 55, 56, 57]
Table 4: [ 1,  3, 13, 16, 23, 24, 35, 52]
Table 5: [ 6, 10, 21, 27, 30, 39, 40, 49]
Table 6: [17, 22, 34, 42, 50, 54, 59, 62]
Table 7: [11, 12, 26, 33, 47, 51, 60, 64]
Table 8: [ 2, 18, 28, 31, 38, 43, 46, 63]
----------------------------------------


## Mutation

- swap mutation <span style="color:red">(from class)</span> <span style="color:orange">(check if greedy is different enough to use)</span> 
- scramble mutation
- inversion mutation <span style="color:red">(from class)</span>
- tableswap mutation

### Greedy Swap Mutation

In [ ]:
# make it genuinly greedy by starting to make personB the max of the ppl with relationships (sort descending)
def greedy_swap_mutation(representation, attendees_num, mut_prob, fitness_grid):
    new_repr = deepcopy(representation)

    # random chance to do the mutation
    if random.random() > mut_prob:
        print("Randomly chosen to not implement.")
        return new_repr

    # selects random different tables to swap from, 
    randomPersonA = randint(1, attendees_num)
    randomTableA = new_repr[randomPersonA-1]

    # get positivley valued neighbors
    filtered = fitness_grid[fitness_grid['idx'] == randomPersonA].iloc[0]
    filtered = filtered.drop('idx')
    peopleWithRelationship = [int(x) for x in filtered[filtered > 0].index] # only getting the attendees that are valued by randomPersonA
    peopleWithRelationship = sorted(peopleWithRelationship, reverse=True)

    # determine person B (don't choose someone from the same table)
    randomPersonB = None
    for personB in peopleWithRelationship:
        randomPersonB = personB
        randomTableB = new_repr[personB-1]
        if randomTableA == randomTableB:
            continue
        else:
            break

    # if there are no values people that don't share the same table as A, randomly choose B
    if randomPersonB == None:
        randomPersonB = randint(1, attendees_num)
        randomTableB = new_repr[randomPersonB-1]
        while randomTableB == randomTableA: # always choose a different table
            randomPersonB = randint(1, attendees_num)
            randomTableB = new_repr[randomPersonB-1]

    # select someone random from tableA thats not randomPersonA
    # get everyone from randomTableA
    tableA_seats = [i for i, x in enumerate(new_repr) if x == randomTableA]
    tableA_seats = [x + 1 for x in tableA_seats] # add 1 to everything to match the fitness grid
    tableA_seats.remove(randomPersonA) # remove personA from the list
    randomPersonC = random.choice(tableA_seats)

    # Swap the two people between the tables
    new_repr[randomPersonC-1] = randomTableB
    new_repr[randomPersonB-1] = randomTableA

    print("Swapped personC #" + str(randomPersonC) + " at table " + str(randomTableA) + " with personB #" + str(personB) + " at table " + str(randomTableB) + " for personA #" + str(randomPersonA))

    return Wedding_Solution(repr=new_repr)

In [17]:
swap_repr = greedy_swap_mutation(repr.repr, repr.attendees, 1, repr.values_grid)
print("Swapped: " + str(swap_repr.repr) + " Fitness: " + str(swap_repr.fitness()))
print("Original: " + str(repr.repr) + " Fitness: " + str(repr.fitness()))

Swapped personC #62 at table 5 with personB #28 at table 3 for personA #16
Swapped: [6, 1, 2, 8, 5, 8, 6, 7, 3, 7, 4, 5, 1, 8, 2, 5, 4, 5, 6, 5, 6, 3, 2, 8, 8, 4, 3, 5, 6, 8, 2, 3, 4, 7, 7, 8, 1, 7, 6, 7, 2, 8, 6, 4, 1, 4, 2, 1, 1, 6, 1, 2, 3, 3, 4, 2, 7, 7, 5, 5, 1, 3, 4, 3] Fitness: 14600
Original: [6, 1, 2, 8, 5, 8, 6, 7, 3, 7, 4, 5, 1, 8, 2, 5, 4, 5, 6, 5, 6, 3, 2, 8, 8, 4, 3, 3, 6, 8, 2, 3, 4, 7, 7, 8, 1, 7, 6, 7, 2, 8, 6, 4, 1, 4, 2, 1, 1, 6, 1, 2, 3, 3, 4, 2, 7, 7, 5, 5, 1, 5, 4, 3] Fitness: 13400


### Scramble Mutation

In [27]:
def scramble_mutation(representation, attendees_num, mut_prob):
    new_repr = deepcopy(representation)

    # random chance to do the mutation
    if random.random() > mut_prob:
        print("Randomly chosen to not implement.")
        return new_repr

    # limit the number of people that can be shuffled to at max 10
    if attendees_num < 10:
        limit = int(attendees_num / 2)
    else:
        limit = 10

    # choose a random number of people to get a subset
    numOfPeopleToShuffle = randint(2, limit) # cant shuffle 1 person, nothing would change

    startPerson = randint(0, attendees_num-numOfPeopleToShuffle) # set the biggest number it can randomly choose as the total - the number aimed to shuffle
    stopPerson = startPerson + numOfPeopleToShuffle
    
    print("Number to shuffle: ", numOfPeopleToShuffle)
    print("Range to shuffle: " + str(startPerson) + " - " + str(stopPerson))

    # shuffle the subset
    peopleToShuffle = new_repr[startPerson:stopPerson]
    random.shuffle(peopleToShuffle)
    new_repr[startPerson:stopPerson] = peopleToShuffle

    return Wedding_Solution(repr=new_repr)

In [40]:
scramble_repr = scramble_mutation(repr.repr, repr.attendees, 1)
print("Scrambled: " + str(scramble_repr.repr) + " Fitness: " + str(scramble_repr.fitness()))
print("Original: " + str(repr.repr) + " Fitness: " + str(repr.fitness()))

Number to shuffle:  7
Range to shuffle: 12 - 19
Scrambled: [6, 1, 2, 8, 5, 8, 6, 7, 3, 7, 4, 5, 5, 6, 4, 5, 1, 2, 8, 5, 6, 3, 2, 8, 8, 4, 3, 3, 6, 8, 2, 3, 4, 7, 7, 8, 1, 7, 6, 7, 2, 8, 6, 4, 1, 4, 2, 1, 1, 6, 1, 2, 3, 3, 4, 2, 7, 7, 5, 5, 1, 5, 4, 3] Fitness: 10700
Original: [6, 1, 2, 8, 5, 8, 6, 7, 3, 7, 4, 5, 1, 8, 2, 5, 4, 5, 6, 5, 6, 3, 2, 8, 8, 4, 3, 3, 6, 8, 2, 3, 4, 7, 7, 8, 1, 7, 6, 7, 2, 8, 6, 4, 1, 4, 2, 1, 1, 6, 1, 2, 3, 3, 4, 2, 7, 7, 5, 5, 1, 5, 4, 3] Fitness: 13400


### Inversion Mutation

In [41]:
def inversion_mutation(representation, attendees_num, mut_prob):
    new_repr = deepcopy(representation)
    
    # random chance to do the mutation
    if random.random() >= mut_prob:
        print("Randomly chosen to not implement.")
        return new_repr
    
    # limit the number of people that can be shuffled to at max 10
    if attendees_num < 10:
        limit = int(attendees_num / 2)
    else:
        limit = 10

    # choose a random number of people to get a subset
    numOfPeopleToInvert = randint(2, limit) # cant invert 1 person, nothing would change - limiting to 10 indivs for less disruption

    startPerson = randint(0, attendees_num-numOfPeopleToInvert) # set the biggest number it can randomly choose as the total - the number aimed to shuffle
    stopPerson = startPerson + numOfPeopleToInvert

    print("Number to invert: ", numOfPeopleToInvert)
    print("Range to invert: " + str(startPerson) + " - " + str(stopPerson))

    # shuffle the subset
    peopleToInvert = new_repr[startPerson:stopPerson]
    print(peopleToInvert)
    peopleToInvert.reverse()
    print(peopleToInvert)
    print("---------")
    new_repr[startPerson:stopPerson] = peopleToInvert
    
    return Wedding_Solution(repr=new_repr)

In [43]:
inversion_repr = inversion_mutation(repr.repr, repr.attendees, 1)
print("Inverted: " + str(inversion_repr.repr) + " Fitness: " + str(inversion_repr.fitness()))
print("Original: " + str(repr.repr) + " Fitness: " + str(repr.fitness()))

Number to invert:  8
Range to invert: 41 - 49
[8, 6, 4, 1, 4, 2, 1, 1]
[1, 1, 2, 4, 1, 4, 6, 8]
---------
Inverted: [6, 1, 2, 8, 5, 8, 6, 7, 3, 7, 4, 5, 1, 8, 2, 5, 4, 5, 6, 5, 6, 3, 2, 8, 8, 4, 3, 3, 6, 8, 2, 3, 4, 7, 7, 8, 1, 7, 6, 7, 2, 1, 1, 2, 4, 1, 4, 6, 8, 6, 1, 2, 3, 3, 4, 2, 7, 7, 5, 5, 1, 5, 4, 3] Fitness: 13900
Original: [6, 1, 2, 8, 5, 8, 6, 7, 3, 7, 4, 5, 1, 8, 2, 5, 4, 5, 6, 5, 6, 3, 2, 8, 8, 4, 3, 3, 6, 8, 2, 3, 4, 7, 7, 8, 1, 7, 6, 7, 2, 8, 6, 4, 1, 4, 2, 1, 1, 6, 1, 2, 3, 3, 4, 2, 7, 7, 5, 5, 1, 5, 4, 3] Fitness: 13400


### TableSwap Mutation

In [96]:
def tableswap_mutation(representation, mut_prob):
    new_repr = deepcopy(representation)

    table_count = max(set(representation))

    # random chance to do the mutation
    if random.random() >= mut_prob:
        print("Randomly chosen to not implement.")
        return new_repr
    
    peopleToSwap = []
    for table_num in range(1, table_count+1):
        table_seats = [i+1 for i, val in enumerate(new_repr) if val == table_num]
        peopleToSwap.append(random.choice(table_seats))

    print(peopleToSwap)

    return None

In [93]:
repr.pretty_print()

----------SEATING ARRANGEMENTS----------
Table 1: [ 2,  6,  9]
Table 2: [ 1,  8, 11]
Table 3: [ 3,  4,  7]
Table 4: [ 5, 10, 12]
----------------------------------------


In [100]:
tableswap_mutation(repr.repr, 1)

[9, 1, 7, 5]


## Crossover

- <s>standard crossover</s> <span style="color:red">not usable</span>
- cycle crossover <span style="color:red">(from class)</span>
- order crossover <span style="color:red">(from chatgpt)</span>

In [7]:
tables = 4
attendees = 16
repr_2 = Wedding_Solution(tables=tables, attendees=attendees)
print(repr_2)
print("Fitness: " + str(repr_2.fitness()))
repr_2.pretty_print()

[2, 3, 3, 4, 4, 1, 2, 3, 1, 2, 2, 3, 4, 1, 1, 4]
Fitness: 3400
----------SEATING ARRANGEMENTS----------
Table 1: [ 6,  9, 14, 15]
Table 2: [ 1,  7, 10, 11]
Table 3: [ 2,  3,  8, 12]
Table 4: [ 4,  5, 13, 16]
----------------------------------------


In [8]:
repr_3 = Wedding_Solution(tables=tables, attendees=attendees)
print(repr_3)
print("Fitness: " + str(repr_3.fitness()))
repr_3.pretty_print()

[1, 1, 1, 2, 4, 2, 4, 2, 1, 3, 3, 2, 4, 3, 3, 4]
Fitness: 8600
----------SEATING ARRANGEMENTS----------
Table 1: [ 1,  2,  3,  9]
Table 2: [ 4,  6,  8, 12]
Table 3: [10, 11, 14, 15]
Table 4: [ 5,  7, 13, 16]
----------------------------------------


### Standard Crossover

> Standard Crossover can't work as the number of tables assigned have to be a certain number

In [ ]:
# def standard_crossover(parent1_repr, parent2_repr):

#     crossoverPoint = randint(1, len(parent1_repr)-1) # cant be 0
#     print("CrossOver Point: " + str(crossoverPoint))
#     child1_repr = parent1_repr[:crossoverPoint] + parent2_repr[crossoverPoint:]
#     child2_repr = parent2_repr[:crossoverPoint] + parent1_repr[crossoverPoint:]

#     print("Child1: ",child1_repr)
#     print("Child2: ",child2_repr)

#     return [Wedding_Solution(repr=child1_repr), Wedding_Solution(repr=child2_repr)]

In [ ]:
# inversion_repr = standard_crossover(repr.repr, repr_2.repr)
# # print(inversion_repr)
# print(repr.repr)
# print(repr_2.repr)
# print("-------")
# print(inversion_repr[0].repr)
# print(inversion_repr[1].repr)

In [ ]:
[1,2,3,4,5] [1,1,2,2,3,3,4,4]

### Cycle Crossover

In [ ]:
def cycle_crossover(parent1_repr, parent2_repr):
    start_idx = randint(0, len(parent1_repr)-1)
    print("Start idx: " + str(start_idx))

    # Initialize the cycle with the starting index
    cycle_idxs = [start_idx]
    current_cycle_idx = start_idx

    tables1_dict = {}
    tables2_dict = {}
    for idx in range(0, len(parent1_repr)):

        # fill the tables per index for repr2
        if parent2_repr[idx] not in tables2_dict.keys():
            tables2_dict[parent2_repr[idx]] = [idx]
        else:
            tables2_dict[parent2_repr[idx]].append(idx)

        # fill the tables per index for repr1
        if parent1_repr[idx] not in tables1_dict.keys():
            tables1_dict[parent1_repr[idx]] = [idx]
        else:
            tables1_dict[parent1_repr[idx]].append(idx)

    # Traverse the cycle by following the mapping from parent2 to parent1
    while True:
        table_parent2 = parent2_repr[current_cycle_idx] #

        # Find where this value is in parent1 to get the next index in the cycle
        next_cycle_idx = random.choice(tables1_dict[table_parent2])
        print("Chose idx #" + str(next_cycle_idx) + " from parent 2.")
        tables1_dict[table_parent2].remove(next_cycle_idx)

        # Closed the cycle -> Break
        if next_cycle_idx == start_idx:
            break

        cycle_idxs.append(next_cycle_idx)
        current_cycle_idx = next_cycle_idx

    offspring1_repr = []
    offspring2_repr = []
    for idx in range(len(parent1_repr)):
        if idx in cycle_idxs:
            # Keep values from parent1 in offspring1 in the cycle indexes
            offspring1_repr.append(parent1_repr[idx])
            # Keep values from parent2 in offspring2 in the cycle indexes
            offspring2_repr.append(parent2_repr[idx])
        else:
            # Swap elements from parents in non-cycle indexes
            offspring1_repr.append(parent2_repr[idx])
            offspring2_repr.append(parent1_repr[idx])

    # To keep the same type as the parents representation
    if isinstance(parent1_repr, str) and isinstance(parent2_repr, str):
        offspring1_repr = "".join(offspring1_repr)
        offspring2_repr = "".join(offspring2_repr)

    return offspring1_repr, offspring2_repr

In [52]:
cycle_repr = cycle_crossover(repr_2.repr, repr_3.repr)
# print(inversion_repr)
print(repr_2.repr)
print(repr_3.repr)
print("-------")
print(cycle_repr[0])
print(cycle_repr[1])

Start idx: 7
Chose idx #6 from parent 2.
Chose idx #3 from parent 2.
Chose idx #0 from parent 2.
Chose idx #5 from parent 2.
Chose idx #9 from parent 2.
Chose idx #2 from parent 2.
Chose idx #13 from parent 2.
Chose idx #7 from parent 2.
[2, 3, 3, 4, 4, 1, 2, 3, 1, 2, 2, 3, 4, 1, 1, 4]
[1, 1, 1, 2, 4, 2, 4, 2, 1, 3, 3, 2, 4, 3, 3, 4]
-------
[2, 1, 3, 4, 4, 1, 2, 3, 1, 2, 3, 2, 4, 1, 3, 4]
[1, 3, 1, 2, 4, 2, 4, 2, 1, 3, 2, 3, 4, 3, 1, 4]


### Order Crossover
<span style="color:red">[CHANGE] needs to be reworded - chatgpt atm</span>
- Picking two random crossover points

- Copying a slice from Parent 1

- Filling in the rest from Parent 2 in order, skipping duplicates

In [55]:
repr_4 = Wedding_Solution(tables=tables, attendees=attendees)
print(repr_4)
print("Fitness: " + str(repr_4.fitness()))
repr_4.pretty_print()
print("----------")
repr_5 = Wedding_Solution(tables=tables, attendees=attendees)
print(repr_5)
print("Fitness: " + str(repr_5.fitness()))
repr_5.pretty_print()

[4, 3, 4, 1, 2, 1, 3, 2, 1, 3, 2, 1, 4, 4, 3, 2]
Fitness: 5900
----------SEATING ARRANGEMENTS----------
Table 1: [ 4,  6,  9, 12]
Table 2: [ 5,  8, 11, 16]
Table 3: [ 2,  7, 10, 15]
Table 4: [ 1,  3, 13, 14]
----------------------------------------
----------
[3, 2, 2, 2, 1, 1, 3, 4, 4, 3, 2, 1, 1, 4, 4, 3]
Fitness: 7600
----------SEATING ARRANGEMENTS----------
Table 1: [ 5,  6, 12, 13]
Table 2: [ 2,  3,  4, 11]
Table 3: [ 1,  7, 10, 16]
Table 4: [ 8,  9, 14, 15]
----------------------------------------


In [ ]:
def order_crossover(parent1_repr, parent2_repr):
    # seats_per_table = parent1_repr.count(1)
    attendee_length = len(parent1_repr)

    # limit the number of people that can be shuffled to at max 10
    if attendee_length < 10:
        limit = int(attendee_length / 2)
    else:
        limit = 10

    # choose a random number of people to get a subset
    numOfPeople = randint(2, limit) # cant invert 1 person, nothing would change - limiting to 10 indivs for less disruption

    startPerson = randint(0, attendee_length-numOfPeople) # set the biggest number it can randomly choose as the total - the number aimed to shuffle
    stopPerson = startPerson + numOfPeople

    print("Number to keep: ", numOfPeople)
    print("Range to keep: " + str(startPerson) + " - " + str(stopPerson))
    print("------------")
    parent1_repr_substring = parent1_repr[startPerson:stopPerson]
    parent2_repr_substring = parent2_repr[startPerson:stopPerson]
    print("Substring 1: " + str(parent1_repr_substring))
    print("Substring 2: " + str(parent2_repr_substring))
    print("------------")
    offspring1_repr, offspring2_repr = [-1] * attendee_length, [-1] * attendee_length
    offspring1_repr[startPerson:stopPerson] = parent1_repr_substring
    offspring2_repr[startPerson:stopPerson] = parent2_repr_substring

    parent1_remaining = deepcopy(parent1_repr)
    parent2_remaining = deepcopy(parent2_repr)

    parent1_remaining.reverse()
    for value in parent2_repr_substring:
        parent1_remaining.remove(value)
    parent1_remaining.reverse()

    parent2_remaining.reverse()
    for value in parent1_repr_substring:
        parent2_remaining.remove(value)
    parent2_remaining.reverse()

    print("Remaining 1: " + str(parent1_remaining))
    print("Remaining 2: " + str(parent2_remaining))

    print("------------")
    offspring_idx = 0
    substring_idx = 0
    while offspring_idx < attendee_length:
        if offspring_idx == startPerson:
            offspring_idx += len(parent1_repr_substring)

        offspring1_repr[offspring_idx] = parent2_remaining[substring_idx]
        offspring2_repr[offspring_idx] = parent1_remaining[substring_idx]
        print("IDX: [" + str(offspring_idx) + "] - " + str(offspring1_repr))

        offspring_idx += 1
        substring_idx += 1

    return offspring1_repr, offspring2_repr

In [77]:
order_repr = order_crossover(repr_2.repr, repr_3.repr)
# print(inversion_repr)
print(repr_2.repr)
print(repr_3.repr)
print("-------")
print(order_repr[0])
print(order_repr[1])

Number to keep:  7
Range to keep: 3 - 10
------------
Substring 1: [4, 4, 1, 2, 3, 1, 2]
Substring 2: [2, 4, 2, 4, 2, 1, 3]
------------
Remaining 1: [2, 3, 3, 4, 4, 1, 3, 1, 1]
Remaining 2: [1, 1, 2, 4, 2, 4, 3, 3, 3]
------------
IDX: [0] - [1, -1, -1, 4, 4, 1, 2, 3, 1, 2, -1, -1, -1, -1, -1, -1]
IDX: [1] - [1, 1, -1, 4, 4, 1, 2, 3, 1, 2, -1, -1, -1, -1, -1, -1]
IDX: [2] - [1, 1, 2, 4, 4, 1, 2, 3, 1, 2, -1, -1, -1, -1, -1, -1]
Length of substring: 7 and idx: 10
IDX: [10] - [1, 1, 2, 4, 4, 1, 2, 3, 1, 2, 4, -1, -1, -1, -1, -1]
IDX: [11] - [1, 1, 2, 4, 4, 1, 2, 3, 1, 2, 4, 2, -1, -1, -1, -1]
IDX: [12] - [1, 1, 2, 4, 4, 1, 2, 3, 1, 2, 4, 2, 4, -1, -1, -1]
IDX: [13] - [1, 1, 2, 4, 4, 1, 2, 3, 1, 2, 4, 2, 4, 3, -1, -1]
IDX: [14] - [1, 1, 2, 4, 4, 1, 2, 3, 1, 2, 4, 2, 4, 3, 3, -1]
IDX: [15] - [1, 1, 2, 4, 4, 1, 2, 3, 1, 2, 4, 2, 4, 3, 3, 3]
[2, 3, 3, 4, 4, 1, 2, 3, 1, 2, 2, 3, 4, 1, 1, 4]
[1, 1, 1, 2, 4, 2, 4, 2, 1, 3, 3, 2, 4, 3, 3, 4]
-------
[1, 1, 2, 4, 4, 1, 2, 3, 1, 2, 4, 2, 4, 3, 3,

# FINISH FOR LATER
[2, 3, 3, 4, 4, 1, 2, 3, 1, 2, 2, 3, 4, 1, 1, 4] <- parent 1
[1, 1, 1, 2, 4, 2, 4, 2, 1, 3, 3, 2, 4, 3, 3, 4] <- parent 2

[4, 4, 1, 2, 3, 1, 2] < will copy from parent 1 to child 1

[1, 1, 1, 2, 4, 2, 4, 2, 1, 3, 3, 2, 4, 3, 3, 4] <- parent 2
[1, 1, _, 2, 4, 2, 4, _, _, 3, 3, _, _, 3, _, _] <- parent 2

[1, 1, 2,                      4, 2, 4, 3, 3, 3] 
[_, _, _, 4, 4, 1, 2, 3, 1, 2, _, _, _, _, _, _] < - offspring 1


In [80]:
order1 = Wedding_Solution(repr=order_repr[0], tables=tables, attendees=attendees)
order1.pretty_print()

----------SEATING ARRANGEMENTS----------
Table 1: [ 1,  2,  6,  9]
Table 2: [ 3,  7, 10, 12]
Table 3: [ 8, 14, 15, 16]
Table 4: [ 4,  5, 11, 13]
----------------------------------------


In [81]:
order2 = Wedding_Solution(repr=order_repr[1], tables=tables, attendees=attendees)
order2.pretty_print()

----------SEATING ARRANGEMENTS----------
Table 1: [ 9, 13, 15, 16]
Table 2: [ 1,  4,  6,  8]
Table 3: [ 2,  3, 10, 14]
Table 4: [ 5,  7, 11, 12]
----------------------------------------
